In [1]:
from tensorflow import keras
import numpy as np
from matplotlib import pyplot as plt
import csv
import cv2

# Download Dataset
Run the codes of this section only one time, it will downlad and store the dataset in your drive

### Mount your drive
First, mount your drive on this notebook. After running the next cell, you can see your google drive directory in Files, the left side of this page.